In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-100k-dataset/ml-100k/u.occupation
/kaggle/input/movielens-100k-dataset/ml-100k/u1.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.info
/kaggle/input/movielens-100k-dataset/ml-100k/u4.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.item
/kaggle/input/movielens-100k-dataset/ml-100k/README
/kaggle/input/movielens-100k-dataset/ml-100k/u1.test
/kaggle/input/movielens-100k-dataset/ml-100k/ua.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.data
/kaggle/input/movielens-100k-dataset/ml-100k/u5.test
/kaggle/input/movielens-100k-dataset/ml-100k/mku.sh
/kaggle/input/movielens-100k-dataset/ml-100k/u5.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.user
/kaggle/input/movielens-100k-dataset/ml-100k/ub.base
/kaggle/input/movielens-100k-dataset/ml-100k/u4.base
/kaggle/input/movielens-100k-dataset/ml-100k/u2.test
/kaggle/input/movielens-100k-dataset/ml-100k/ua.base
/kaggle/input/movielens-100k-dataset/ml-100k/u3.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.

In [3]:

# Specify the file path to your MovieLens 100k data
file_path = '/kaggle/input/movielens-100k-dataset/ml-100k/u.data'

# Load the data into a pandas DataFrame
columns = ['User_id', 'Movie_id', 'Rating', 'Timestamp']
df = pd.read_csv(file_path, names=columns, delimiter='\t')  # Use delimiter=',' for CSV files

# Omit the Timestamp column
df = df[['User_id', 'Movie_id', 'Rating']]

# Display the first few rows of the DataFrame
print(df.head())


   User_id  Movie_id  Rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1


In [4]:
from sklearn.model_selection import train_test_split


X = df[['User_id', 'Movie_id']]
y = df['Rating']

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting train and test sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (80000, 2)
Shape of X_test: (20000, 2)
Shape of y_train: (80000,)
Shape of y_test: (20000,)


In [5]:
# Assuming you have the train and test sets (X_train, X_test, y_train, y_test)

# Create dense training data table
training_data = pd.DataFrame(index=X_train['User_id'].unique(), columns=X_train['Movie_id'].unique())
training_data = training_data.sort_index(axis=0).sort_index(axis=1)  # Sort indices for consistency

# Fill in the ratings from the training set
for i, row in X_train.iterrows():
    user_id, movie_id, rating = row['User_id'], row['Movie_id'], y_train[i]
    training_data.at[user_id, movie_id] = rating


# Create dense testing data table
testing_data = pd.DataFrame(index=X_test['User_id'].unique(), columns=X_test['Movie_id'].unique())
testing_data = testing_data.sort_index(axis=0).sort_index(axis=1)  # Sort indices for consistency

# Fill in the ratings from the testing set
for i, row in X_test.iterrows():
    user_id, movie_id, rating = row['User_id'], row['Movie_id'], y_test[i]
    testing_data.at[user_id, movie_id] = rating


In [10]:
# Fill missing values with 0 or another appropriate value
training_data = training_data.fillna(0)


# Display the dense training data table
print("Dense Training Data Table:")
print(training_data.head())

# Fill missing values with 0 or another appropriate value
testing_data = testing_data.fillna(0)

# Display the dense testing data table
print("\nDense Testing Data Table:")
print(testing_data.head())

Dense Training Data Table:
   1     2     3     4     5     6     7     8     9     10    ...  1668  \
1     0     3     4     0     3     0     4     0     5     3  ...     0   
2     4     0     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     0     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   
5     4     0     0     0     0     0     0     0     0     0  ...     0   

   1670  1671  1672  1673  1676  1678  1679  1680  1681  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  
5     0     0     0     0     0     0     0     0     0  

[5 rows x 1653 columns]

Dense Testing Data Table:
   1     2     3     4     5     6     7     8     9     10    ...  1648  \
1     5     0     0     3     0     5   

In [15]:
# Assuming you have the dense training data table (training_data)

# Calculate the global mean
global_mean = training_data.values[training_data.values != 0].mean()

print("Global Mean of Ratings in the Training Data:", global_mean)


Global Mean of Ratings in the Training Data: 3.5312625


In [19]:
# Calculate items' bias (bi) for each item i
items_bias = {}
# movie_columns = training_data.columns
# print(movie_columns.head())
for item_id in movie_columns:
    item_ratings = training_data[item_id][training_data[item_id] != 0]  # Select non-zero ratings
    num_ratings = len(item_ratings)
    if num_ratings > 0:
        bi = (item_ratings - global_mean).sum() / num_ratings
        items_bias[item_id] = bi
    else:
        items_bias[item_id] = 0  # Handle the case where the item has no ratings

# Create a new column 'itemBias' in the training data
training_data['itemBias'] = items_bias

# Display the training data with the 'itemBias' column
print("Training Data with Item Bias:")
print(training_data.head())


Training Data with Item Bias:
   1  2  3  4  5  6  7  8  9  10  ...  1670  1671  1672  1673  1676  1678  \
1  0  3  4  0  3  0  4  0  5   3  ...     0     0     0     0     0     0   
2  4  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
3  0  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
4  0  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
5  4  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   

   1679  1680  1681  itemBias  
1     0     0     0  0.334067  
2     0     0     0 -0.327379  
3     0     0     0 -0.429813  
4     0     0     0  0.017870  
5     0     0     0 -0.284886  

[5 rows x 1654 columns]


/tmp/ipykernel_41/563120244.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  training_data['itemBias'] = items_bias


In [33]:
# Assuming you have the dense training data table (training_data), global_mean, and users_bias calculated earlier

# Calculate items' bias (bi) for each item i
items_bias = {}
# traning_data_exclude = training_data.columns[:-2]
print(training_data.head())
# for item_id in training_data.columns[:-2]:  # Exclude the 'userBias' and 'itemBias' columns
#     item_ratings = training_data[item_id][training_data[item_id] != 0]  # Select non-zero ratings
#     num_ratings = len(item_ratings)
#     if num_ratings > 0:
#         bi = (item_ratings - (global_mean + training_data['userBias'])).sum() / num_ratings
#         items_bias[item_id] = bi
#     else:
#         items_bias[item_id] = 0  # Handle the case where the item has no ratings

# # Create a new column 'itemBias' in the training data
# training_data['itemBias'] = training_data.columns[:-2].map(items_bias.get)

# # Display the training data with the 'itemBias' column
# print("Training Data with Item Bias:")
# print(training_data.head())


   1  2  3  4  5  6  7  8  9  10  ...  1670  1671  1672  1673  1676  1678  \
1  0  3  4  0  3  0  4  0  5   3  ...     0     0     0     0     0     0   
2  4  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
3  0  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
4  0  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
5  4  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   

   1679  1680  1681  userBias  
1     0     0     0  0.156237  
2     0     0     0  0.273085  
3     0     0     0 -0.731263  
4     0     0     0  0.968737  
5     0     0     0 -0.668944  

[5 rows x 1654 columns]


In [20]:
# Calculate users' bias (bu) for each user u
users_bias = {}
for user_id, row in training_data.iterrows():
    user_ratings = row[:-1]  # Exclude 'itemBias' column
    rated_items = user_ratings[user_ratings != 0]
    
    # Calculate the number of items rated by user u and adjust for a small constant
    num_rated_items = 0.99 + abs(len(rated_items))
    
    if num_rated_items > 0:
        bu = (rated_items - (global_mean + training_data['itemBias'])).sum() / num_rated_items
        users_bias[user_id] = bu
    else:
        users_bias[user_id] = 0  # Handle the case where the user has no ratings

# Create a new column 'userBias' in the training data
training_data['userBias'] = training_data.index.map(users_bias)

# Display the training data with the 'userBias' column
print("Training Data with User Bias:")
print(training_data.head())



Training Data with User Bias:
   1  2  3  4  5  6  7  8  9  10  ...  1671  1672  1673  1676  1678  1679  \
1  0  3  4  0  3  0  4  0  5   3  ...     0     0     0     0     0     0   
2  4  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
3  0  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
4  0  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   
5  4  0  0  0  0  0  0  0  0   0  ...     0     0     0     0     0     0   

   1680  1681  itemBias  userBias  
1     0     0  0.334067  0.134723  
2     0     0 -0.327379  0.199249  
3     0     0 -0.429813 -0.493094  
4     0     0  0.017870  1.044080  
5     0     0 -0.284886 -0.353129  

[5 rows x 1655 columns]


/tmp/ipykernel_41/3943886886.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  training_data['userBias'] = training_data.index.map(users_bias)


In [26]:
# Create a copy of the training data to store the filled values
training_data_filled = training_data.copy()

# Iterate through each row
for user_id, row in training_data.iterrows():
    for movie_id in training_data.columns[:-3]:
        # Check if the value is missing (original value is 0)
        if row[movie_id] == 0:
            # Update the missing value using the formula: missing(u, i) = bu + bi + global_mean
            filled_value = users_bias[user_id] + items_bias[movie_id] + global_mean
            training_data_filled.at[user_id, movie_id] = filled_value

# Display the filled training data
print("Filled Training Data:")
print(training_data_filled.head())

Filled Training Data:
          1         2         3         4         5         6         7  \
1  4.000053  3.000000  4.000000  3.683856  3.000000  3.781782  4.000000   
2  4.000000  3.403133  3.300699  3.748382  3.445626  3.846308  4.011162   
3  3.372236  2.710790  2.608355  3.056039  2.753283  3.153965  3.318818   
4  4.909410  4.247964  4.145529  4.593213  4.290457  4.691139  4.855992   
5  4.000000  2.850755  2.748320  3.196004  2.893248  3.293930  3.458783   

          8         9        10  ...      1671      1672      1673      1676  \
1  4.070770  5.000000  3.000000  ...  1.134723  2.134723  3.134723  2.134723   
2  4.135296  4.107199  4.091141  ...  1.199249  2.199249  3.199249  2.199249   
3  3.442953  3.414856  3.398798  ...  0.506906  1.506906  2.506906  1.506906   
4  4.980127  4.952030  4.935972  ...  2.044080  3.044080  4.044080  3.044080   
5  3.582918  3.554821  3.538763  ...  0.646871  1.646871  2.646871  1.646871   

       1678      1679      1680  1681  itemBia

In [38]:
from sklearn.decomposition import TruncatedSVD

# Fill NaN values with a specific value (e.g., 0)
training_data_filled = training_data_filled.fillna(0)

# Specify the number of components (you can adjust this based on your requirements)
n_components = min(training_data_filled.shape) - 1

# Create TruncatedSVD instance and fit_transform the filled training data
svd = TruncatedSVD(n_components=n_components)
U = svd.fit_transform(training_data_filled)
S = np.diag(svd.singular_values_)
V = svd.components_

# Display the shapes of U, S, and V
print("Shape of U:", U.shape)
print("Shape of S:", S.shape)
print("Shape of V:", V.shape)


Shape of U: (943, 942)
Shape of S: (942, 942)
Shape of V: (942, 1655)


In [39]:
# Specify the number of components (you can adjust this based on your requirements)
n_components = 10  # Replace 10 with the desired number of components

# Reduce U, S, and V
U_approx = U[:, :n_components]
S_approx = S[:n_components, :n_components]
V_approx = V[:n_components, :]

# Display the shapes of the reduced matrices
print("Shape of U_approx:", U_approx.shape)
print("Shape of S_approx:", S_approx.shape)
print("Shape of V_approx:", V_approx.shape)


Shape of U_approx: (943, 10)
Shape of S_approx: (10, 10)
Shape of V_approx: (10, 1655)


In [40]:
# Calculate Z
Z_approx = np.dot(U_approx, np.dot(S_approx, V_approx))

# Display the shape of Z
print("Shape of Z_approx:", Z_approx.shape)


Shape of Z_approx: (943, 1655)


In [ ]:
# Assuming testing_data and Z_approx are NumPy arrays
non_zero_mask = testing_data != 0
mae_masked = np.mean(np.abs(testing_data[non_zero_mask] - Z_approx[non_zero_mask]))
print("Mean Absolute Error (MAE) for non-zero values:", mae_masked)